In [1]:
## Adaptive Testing 

import openai
from openai import OpenAI
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:

os.environ["OPENAI_API_KEY"] = "sk-proj-fPbd82pXUd0snI5DTCSYT3BlbkFJNT1ZqfvGxFa1K1Pz3bJG"
def generate_questions(prompt, num_questions=10):
    try:
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise ValueError("OPENAI_API_KEY environment variable not set.")

        client = OpenAI(api_key=api_key)
        generated_texts = set()
        questions = []
        i = 0

        while i < num_questions:
            variation_prompt = f"{prompt} - Question {i+1}"

            response = client.completions.create(
                model="gpt-3.5-turbo-instruct",
                prompt=f"Generate a multiple-choice question about {variation_prompt} strictly in the format:\n\n" \
                       f"Question:\n" \
                       f"Option A:\n" \
                       f"Option B:\n" \
                       f"Option C:\n" \
                       f"Option D:\n" \
                       f"Correct Option: Option A or Option B or Option C or Option D",
                max_tokens=150,
                n=1,
                stop=None,
                temperature=0.9
            )

            question_text = response.choices[0].text.strip()

            if question_text in generated_texts:
                continue

            generated_texts.add(question_text)

            lines = question_text.splitlines()
            lines = [line.strip() for line in lines if line.strip()]

            if not (lines[0].startswith('Question') and lines[1].startswith('Option A') and
                    lines[2].startswith('Option B') and lines[3].startswith('Option C') and
                    lines[4].startswith('Option D') and lines[5].startswith('Correct')):
                continue

            question_data = {
                "Question": lines[0].replace("Question:", "").strip(),
                "Category": prompt,
                "Option A": lines[1].replace("Option A:", "").strip(),
                "Option B": lines[2].replace("Option B:", "").strip(),
                "Option C": lines[3].replace("Option C:", "").strip(),
                "Option D": lines[4].replace("Option D:", "").strip(),
                "Correct Option": lines[5].replace("Correct Option:", "").strip()[0:8],
                'difficulty_index': np.random.uniform(0.1, 1.0),
                'discriminatory_index': np.random.uniform(0.1, 1.0),
            }

            questions.append(question_data)
            i += 1

        questions_df = pd.DataFrame(questions)
        return questions_df

    except ValueError as ve:
        print(f"ValueError: {ve}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None



In [18]:
# Prompts to generate questions
prompts = ["Biology", "Psychology", "Gynaecology", "Orthopedic","Oncology"]
questions_df_list = [generate_questions(prompt, num_questions=10) for prompt in prompts]
all_questions_df = pd.concat(questions_df_list, ignore_index=True)

if all_questions_df is not None:
    print("Generated Questions:")
    print(all_questions_df)
else:
    print("Failed to generate questions.")

all_questions_df=all_questions_df.reset_index().rename(columns={'index':'question_id'})
all_questions_df.to_csv('Questions.csv',header=True)


Generated Questions:
                                             Question     Category  \
0   What is the process by which plants convert su...      Biology   
1   Which of the following is the basic unit of li...      Biology   
2   Which of the following organelles is responsib...      Biology   
3   Which of the following is not a type of blood ...      Biology   
4   Which of the following is a characteristic of ...      Biology   
5   What is the function of a mitochondrion in a c...      Biology   
6   Which of the following is NOT a function of th...      Biology   
7   What is the process by which genetic informati...      Biology   
8   Which of the following is the primary function...      Biology   
9   What is the process by which plants use sunlig...      Biology   
10  Which of the following is NOT a primary approa...   Psychology   
11   What is the main focus of humanistic psychology?   Psychology   
12  What is the primary focus of cognitive psychol...   Psychology   

In [20]:

questions_list = all_questions_df.to_dict(orient='records')

In [23]:
import gurobipy as gp
from gurobipy import GRB
import ipywidgets as widgets
from IPython.display import display, clear_output
from functools import partial

class AdaptiveTest:
    def __init__(self, questions, total_questions=5):
        self.questions = questions
        self.total_questions = total_questions
        self.asked_questions = []
        self.correct_answers = 0
        self.correct_difficulty_indices = []
        self.current_difficulty = 0.5  # Starting difficulty index
        self.categories = list(set(q['category'] for q in questions))

    def adjust_difficulty(self, previous_difficulty, correct):
        if correct:
            return min(1.0, previous_difficulty + 0.1)
        else:
            return max(0.1, previous_difficulty - 0.1)

    def setup_milp_model(self, available_questions, category_counts):
        model = gp.Model("Question_Selection")
        
        # Variables: x[i] is 1 if question i is selected, 0 otherwise
        x = model.addVars(len(available_questions), vtype=GRB.BINARY, name="x")
        
        # Objective: Maximize the discriminatory index
        model.setObjective(gp.quicksum(available_questions[i]['discriminatory_index'] * x[i] for i in range(len(available_questions))), GRB.MAXIMIZE)
        
        # Constraints: Ensure 20% ± 5% category representation for the remaining questions
        remaining_questions = self.total_questions - len(self.asked_questions)
        for cat in self.categories:
            category_indices = [i for i in range(len(available_questions)) if available_questions[i]['category'] == cat]
            min_cat = max(0, 0.15 * remaining_questions - category_counts[cat])
            max_cat = min(len(category_indices), 0.25 * remaining_questions - category_counts[cat])
            model.addConstr(gp.quicksum(x[i] for i in category_indices) >= min_cat)
            model.addConstr(gp.quicksum(x[i] for i in category_indices) <= max_cat)
        
        # Ensure only one question is selected
        model.addConstr(gp.quicksum(x[i] for i in range(len(available_questions))) == 1)
        
        return model, x

    def get_next_question(self):
        available_questions = [q for q in self.questions if q['question_id'] not in self.asked_questions and 
                               abs(q['difficulty_index'] - self.current_difficulty) <= 0.2]

        if not available_questions:
            print("No more questions available with the current difficulty settings.")
            return None

        # Select a question ensuring 20% ± 5% category representation
        category_counts = {cat: 0 for cat in self.categories}
        for q_id in self.asked_questions:
            category_counts[self.questions[q_id]['category']] += 1

        model, x = self.setup_milp_model(available_questions, category_counts)
        model.optimize()
        
        selected_question_indices = [i for i in range(len(available_questions)) if x[i].x > 0.5]
        
        if not selected_question_indices:
            print("No feasible solution found by the MILP model. Relaxing constraints.")
            self.current_difficulty = self.adjust_difficulty(self.current_difficulty, False)
            return self.get_next_question()
        
        selected_question_index = selected_question_indices[0]
        selected_question = available_questions[selected_question_index]
        self.asked_questions.append(selected_question['question_id'])
        
        return selected_question

    def ask_question(self, question):
        clear_output(wait=True)  # Clear output before printing question
        print(f"Question: {question['question_id']}, Category: {question['category']}")  # Print the question
        option_buttons = []
        for idx, option in enumerate([question['options'][0], question['options'][1], question['options'][2], question['options'][3]]):
            button = widgets.Button(description=f"{chr(65+idx)}: {option}")
            button.on_click(partial(self.check_answer, question, chr(65+idx)))
            option_buttons.append(button)
            button.layout.width = 'auto'
            display(button)  # Display the options

    def check_answer(self, question, answer, b):
        correct_option_letter = chr(65 + question['correct_option'])  # Get the letter (A, B, C, or D)
        correct = (answer == correct_option_letter) 
        if correct:
            self.correct_answers += 1
            self.correct_difficulty_indices.append(question['difficulty_index'])
        
        self.current_difficulty = self.adjust_difficulty(self.current_difficulty, correct)
        
        if len(self.asked_questions) < self.total_questions:
            next_question = self.get_next_question()
            if next_question:
                self.ask_question(next_question)
            else:
                self.finish_test()
        else:
            self.finish_test()

    def finish_test(self):
        clear_output(wait=True)
        final_score = sum(self.correct_difficulty_indices)
        print(f"Test completed! You answered {self.correct_answers} out of {self.total_questions} questions correctly.")
        print(f"Your final score is: {final_score:.2f}")

    def start_test(self):
        self.correct_answers = 0
        self.asked_questions = []
        self.correct_difficulty_indices = []
        self.current_difficulty = 0.5  # Reset starting difficulty
        first_question = self.get_next_question()
        if first_question:
            self.ask_question(first_question)

# Example question list
questions_list = [{
    'question_id': i,
    'options': [f'Option {j+1}' for j in range(4)],
    'difficulty_index': np.random.uniform(0.1, 1.0),
    'discriminatory_index': np.random.uniform(0.1, 1.0),
    'category': np.random.choice(['Math', 'Science', 'History', 'Geography']),
    'correct_option': np.random.randint(0, 4)
} for i in range(1000)]

# Initialize the AdaptiveTest class with the questions
test = AdaptiveTest(questions_list)
test.start_test()


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[rosetta2] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9 rows, 428 columns and 1284 nonzeros
Model fingerprint: 0x03b33130
Variable types: 0 continuous, 428 integer (428 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 1e+00]
Presolve removed 0 rows and 424 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible
Best objective -, best bound -, gap -


AttributeError: Unable to retrieve attribute 'x'

In [22]:
import pulp
import ipywidgets as widgets
from IPython.display import display, clear_output
from functools import partial

class AdaptiveTest:
    def __init__(self, questions, total_questions=5):
        self.questions = questions
        self.total_questions = total_questions
        self.asked_questions = []
        self.correct_answers = 0
        self.correct_difficulty_indices = []
        self.current_difficulty = 0.5  # Starting difficulty index
        self.categories = list(set(q['Category'] for q in questions))

    def adjust_difficulty(self, previous_difficulty, correct):
        if correct:
            return min(1.0, previous_difficulty + 0.1)
        else:
            return max(0.1, previous_difficulty - 0.1)

    def setup_milp_model(self, available_questions, category_counts):
        model = pulp.LpProblem("Question_Selection", pulp.LpMaximize)
        x = pulp.LpVariable.dicts("x", range(len(available_questions)), cat="Binary")
        
        # Objective: Maximize the discriminatory index
        model += pulp.lpSum([available_questions[i]['discriminatory_index'] * x[i] for i in range(len(available_questions))])
        
        # Constraints: Ensure 20% ± 5% category representation for the remaining questions
        remaining_questions = self.total_questions - len(self.asked_questions)
        for cat in self.categories:
            category_indices = [i for i in range(len(available_questions)) if available_questions[i]['Category'] == cat]
            min_cat = max(0, 0.15 * remaining_questions - category_counts[cat])
            max_cat = min(len(category_indices), 0.25 * remaining_questions - category_counts[cat])
            #model += pulp.lpSum([x[i] for i in category_indices]) >= min_cat
            #model += pulp.lpSum([x[i] for i in category_indices]) <= max_cat
        
        # Ensure only one question is selected
        model += pulp.lpSum([x[i] for i in range(len(available_questions))]) == 1
        
        return model, x

    def get_next_question(self):
        available_questions = [q for q in self.questions if q['question_id'] not in self.asked_questions and 
                               abs(q['difficulty_index'] - self.current_difficulty) <= 0.2]

        if not available_questions:
            print("No more questions available with the current difficulty settings.")
            return None

        # Select a question ensuring 20% ± 5% category representation
        category_counts = {cat: 0 for cat in self.categories}
        for q_id in self.asked_questions:
            category_counts[self.questions[q_id]['Category']] += 1

        model, x = self.setup_milp_model(available_questions, category_counts)
        # **MISSING LINE:** Solve the MILP model
        model.solve() 
        
        selected_question_indices = [i for i in range(len(available_questions)) if pulp.value(x[i]) == 1]
        
        if not selected_question_indices:
            print("No feasible solution found by the MILP model. Relaxing constraints.")
            self.current_difficulty = self.adjust_difficulty(self.current_difficulty, False)
            return self.get_next_question()
        
        selected_question_index = selected_question_indices[0]
        selected_question = available_questions[selected_question_index]
        self.asked_questions.append(selected_question['question_id'])
        
        return selected_question

    def ask_question(self, question):
        clear_output(wait=True)  # Clear output before printing question
        print(f"Question: {question['Question']}, Category: {question['Category']}")  # Print the question
        option_buttons = []
        for idx, option in enumerate([question['Option A'], question['Option B'], question['Option C'], question['Option D']]):
            button = widgets.Button(description=f"{chr(65+idx)}: {option}")
            button.on_click(partial(self.check_answer, question, chr(65+idx)))
            option_buttons.append(button)
            button.layout.width = 'auto'
            display(button)  # Display the options

    def check_answer(self, question, answer, b):
        correct_option_letter = question['Correct Option'][-1]  # Get the last character (A, B, C, or D)
        correct = (answer == correct_option_letter) 
        if correct:
            self.correct_answers += 1
            self.correct_difficulty_indices.append(question['difficulty_index'])
        
        self.current_difficulty = self.adjust_difficulty(self.current_difficulty, correct)
        
        if len(self.asked_questions) < self.total_questions:
            next_question = self.get_next_question()
            if next_question:
                self.ask_question(next_question)
            else:
                self.finish_test()
        else:
            self.finish_test()

    def finish_test(self):
        clear_output(wait=True)
        final_score = sum(self.correct_difficulty_indices)
        print(f"Test completed! You answered {self.correct_answers} out of {self.total_questions} questions correctly.")
        print(f"Your final score is: {final_score:.2f}")

    def start_test(self):
        self.correct_answers = 0
        self.asked_questions = []
        self.correct_difficulty_indices = []
        self.current_difficulty = 0.5  # Reset starting difficulty
        first_question = self.get_next_question()
        if first_question:
            self.ask_question(first_question)

# Initialize the AdaptiveTest class with the questions
test = AdaptiveTest(questions_list)
test.start_test()



Test completed! You answered 4 out of 5 questions correctly.
Your final score is: 2.21
